# Exoplanet Transit Interpretation code (EXOTIC)

### <font color='red'>This is YOUR copy of"</font>the NASA EXOTIC Jupyter Notebook. 

Then run the cells of this notebook in sequence by clicking the "play" button at the left of each code cell.  [Here](https://www.youtube.com/watch?v=PwPDttan7KQ) is a screencast that walks you through the process with the sample data (view at high-resolution).  EXOTIC will use our JupyterHub Bushastrolab virtual machine in the cloud to run the code, but it will use images in your JupyterHub account as input. 

Most of the code that is used for the steps in this Jupyter Notebook has been hidden, but if you want to see it, double-click on the corresponding code cell.  (To re-hide Press Shift-Enter.)


In [1]:
#!pip list


In [2]:
#@title <font color='red'>Click "run"</font> (at left) to install EXOTIC and give it permission to run on your Google drive files.
#%%capture
#!pip install exotic --upgrade
import bokeh.io
# from exotic.api.plotting import plot_image
from exotic.exotic import NASAExoplanetArchive, get_wcs, find_target
from astropy.time import Time
from barycorrpy import utc_tdb
import numpy as np
from io import BytesIO
from astropy.io import fits
from scipy.ndimage import label
from bokeh.plotting import figure, output_file, show
from bokeh.palettes import Viridis256
from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper, LogTicker
from bokeh.models import BoxZoomTool,WheelZoomTool,ResetTool,HoverTool,PanTool,FreehandDrawTool
from bokeh.io import output_notebook
from bokeh.layouts import row
from pprint import pprint
from IPython.display import Image
from ipywidgets import widgets, HBox
from skimage.transform import rescale, resize, downscale_local_mean
import copy
import matplotlib.pyplot as plt
#!pip install dynesty
import dynesty
from dynesty import plotting
from dynesty.utils import resample_equal
from scipy.stats import gaussian_kde
import os
import re
import json
import subprocess
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

    Done!
Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...


Wait until the cell above is finished running before proceeding.  You will know that it is done running because the Colaboratory logo at the top of the tab turns from gray to yellow.  (In general, a gray Colaboratory logo means "running", yellow means "ready for the next step", and red means "something is wrong".)  Note that there is an incrementing timer plus "headline" command strings scrolling across the status bar at the bottom of the browser window; these are replaced with a total run time and date stamp on completion of the cell. 

Once it is done, look up your target in the NASA Exoplanet Archive.  In order to fit a lightcurve with appropriate priors, EXOTIC needs to know when the planet is expected to transit and how much starlight the planet is expected to block when it does.  (If you know what an inits file is and you already have one, then you do not *need* to run this target lookup cell, but it is always a good check.)

If you are running the sample data, then your target is Hat-p-32 b.  


In [3]:
#@title <font color='red'>Click "run"</font> to look up your target in the NASA Exoplanet Archive.

##############################################################

def fix_planetary_params (p_param_dict):
  for param in p_param_dict.keys():
    if param == "Target Star RA" or param == "Target Star Dec" or param == "Planet Name" or param == "Host Star Name" or param == "Argument of Periastron (deg)":
      continue
    val = p_param_dict[param]
    if val == 0.0 or np.isnan(float(val)):
      if param == "Orbital Eccentricity (0 if null)":
        continue
      if param == "Ratio of Planet to Stellar Radius (Rp/Rs)":
        p_param_dict[param] = 0.151
      if param == "Ratio of Planet to Stellar Radius (Rp/Rs) Uncertainty":
        p_param_dict[param] = 0.151
        if p_param_dict["Host Star Name"] == "Qatar-6":
          p_param_dict[param] = 0.01
      print(f"\nIn the planetary parameters from the NASA Exoplanet Archive, \n\"{param}\" is listed as {val}.\n\n**** This might make EXOTIC crash. ****\n\nIf the parameter is *not* changed below, please edit it\nin the inits file before running EXOTIC.\n")
  p_param_string = json.dumps(p_param_dict)

  planetary_params = "\"planetary_parameters\": {\n"
  num_done, num_total = 0, len(p_param_dict.keys())
  for key, value in p_param_dict.items():
    num_done += 1
    if key == "Target Star RA" or key == "Target Star Dec" or key == "Planet Name" or key == "Host Star Name":
      planetary_params = planetary_params + str(f"    \"{key}\": \"{value}\",\n")
    else:
      if num_done < num_total:
        planetary_params = planetary_params + str(f"    \"{key}\": {value},\n")
      else:
        planetary_params = planetary_params + str(f"    \"{key}\": {value}\n")
  planetary_params = planetary_params + "}"

  return(planetary_params)

##############################################################

print("YOU NEED TO ENTER THE NAME OF YOUR EXOPLANET TARGET OR ENTER THE SAMPLE TARGET")
print("CHOICES FOR BUSH EXOPLANET TARGETS ARE 'WASP-80b' OR 'WASP-135 b' OR 'QATAR-4 b' OR 'HAT P-53 b OR ENTER THE SAMPLE TARGET Hat-p-32 b")
print("DO NOT IGNORE THIS STEP !!!!")

target=input("Please enter the name of your exoplanet target: ")

targ = NASAExoplanetArchive(planet=target)
target = targ.planet_info()[0]
planetary_params = ""
if not targ.resolve_name():
  print("Sorry, can't find your target in the Exoplanet Archive.  Unfortunately, this")
  print("isn't going to work until I can find it.  Please re-run this cell, trying")
  print("different formats for your target name, until the target is located.")
  print("Looking it up in the NASA Exoplanet Archive at https://exoplanetarchive.ipac.caltech.edu/")
  print("might help you know where to put the spaces and hyphens and such.")
else:
  p_param_string = targ.planet_info(fancy=True)
  planetary_params = "\"planetary_parameters\": "+p_param_string
  p_param_dict = json.loads(p_param_string)
  planetary_params = fix_planetary_params(p_param_dict)
  print(f"\nNASA Exoplanet Archive planetary parameters for {target}:\n")
  print(planetary_params)


YOU NEED TO ENTER THE NAME OF YOUR EXOPLANET TARGET OR ENTER THE SAMPLE TARGET
CHOICES FOR BUSH EXOPLANET TARGETS ARE 'WASP-80b' OR 'WASP-135 b' OR 'QATAR-4 b' OR 'HAT P-53 b OR ENTER THE SAMPLE TARGET Hat-p-32 b
DO NOT IGNORE THIS STEP !!!!


Please enter the name of your exoplanet target:  Hat-P 32 b



Looking up HAT-P-32 b on the NASA Exoplanet Archive. Please wait....
Successfully found HAT-P-32 b in the NASA Exoplanet Archive!

NASA Exoplanet Archive planetary parameters for HAT-P-32 b:

"planetary_parameters": {
    "Target Star RA": "02:04:10.262736",
    "Target Star Dec": "+46:41:16.26432",
    "Planet Name": "HAT-P-32 b",
    "Host Star Name": "HAT-P-32",
    "Orbital Period (days)": 2.1500079,
    "Orbital Period Uncertainty": 1.4e-07,
    "Published Mid-Transit Time (BJD-UTC)": 2456628.505625,
    "Mid-Transit Time Uncertainty": 7.1e-05,
    "Ratio of Planet to Stellar Radius (Rp/Rs)": 0.14886235252742716,
    "Ratio of Planet to Stellar Radius (Rp/Rs) Uncertainty": 0.0005539487393037134,
    "Ratio of Distance to Stellar Radius (a/Rs)": 5.344,
    "Ratio of Distance to Stellar Radius (a/Rs) Uncertainty": 0.039496835316262996,
    "Orbital Inclination (deg)": 88.98,
    "Orbital Inclination (deg) Uncertainty": 0.7602631123499285,
    "Orbital Eccentricity (0 if null)": 0.1

### Specifying the path to your images:
* If you are a new user running the Hat-p-32b sample data, just press enter. This will download the sample data to your Google drive account, ask you to identify the target and pick some comparison stars.  The target star is at the center of [this starchart](https://app.aavso.org/vsp/chart/?star=HAT-P-32&scale=F&orientation=visual&type=chart&fov=18.5&maglimit=18.5&resolution=150&north=down&east=left), labeled with an empty circle.  AAVSO recomends using the numbered stars as comparison stars.  Once you find the stars in your image, EXOTIC will fit a lightcurve and save the output files in your Google drive account. 
* If you are reducing a single transit, enter the path to the google drive folder that holds your images. On your computer in browser
it will look like, for example: My Drive > exotransits > Wasp-43. When asked for the folder, you would enter "exotransits/Wasp-43."  Use the folder icon on the left menu bar to locate the folder, right-click on it, and select "Copy path".  If you have them, calibration images should be in folders named "dark", "flat", and "bias" *within* the folder containing your images.  The first image of the series will be displayed so that you can identify your target and pick some comparison stars (see [here](https://app.aavso.org/vsp/) for AAVSO recommended comps).  You will also be asked to specify any needed parameters that EXOTIC cannot find from your image headers.
* If you know what an initialization file is and you already have one, put it into the folder with your images and enter the folder name as specified above.  If there is *exactly one* file with a name ending in .json in the folder with your images, this will automatically be used to initialize EXOTIC and reduce your images.
* If you have two or more initialization files and you want to reduce multiple transits, enter the path to a folder that only contains your inits files.  

In [4]:
#@title <font color='red'>Click "run"</font> to start the data reduction.
# If the user presses enter to run the sample data, download sample data if needed and
# put it into a sample-data directory at the top level of the user's Gdrive.  Count
# the .fits files (images) and .json files (inits files) in the directory entered 
# by the user (or in the sample-data directory if the user pressed enter).  If 
# there are at least 20 .fits files, assume this is a directory of images and display
# the first one in the series.  If there is exactly one inits file in the directory, 
# show the specified target and comp coords so that the user can check these against
# the displayed image.  Otherwise, prompt for target / comp coords and make an inits 
# file based  on those (save this new inits file in the folder with the output files 
# so that the student can consult it later).  Finally, run EXOTIC with the newly-made 
# or pre-existing inits file, plus any other inits files in the directory.

#########################################################
def display_image(filename):
    print(f"{filename}")
    hdu = fits.open(filename)

# replace 0's with extension      # Stuff to put in if the hdul ever becomes a problem.    
#    extension = 0
#    image_header = hdul[extension].header
#    while image_header["NAXIS"] == 0:
#      extension += 1
#      image_header = hdul[extension].header

    dheader = dict(hdu[0].header)
  
    data = hdu[0].data
    megapixel_factor = (data.shape[0])*(data.shape[1])/1000000.0
    if megapixel_factor > 5:
      print(f"Downsampling image because it has {megapixel_factor} megapixels.")
      image_downscaled = downscale_local_mean(data, (2, 2)).astype(int)
      data = image_downscaled
    
    max_y = len(data)
    max_x = len(data[0])
    p_height = 500
    p_width = int((p_height/max_y) * max_x)

    # quick hot pixel/ cosmic ray mask
    # mask, cdata = detect_cosmics(
    #     data, psfmodel='gauss',
    #     psffwhm=4, psfsize=2*round(4)+1, # just a guess
    #     sepmed=False, sigclip = 4.25,
    #     niter=3, objlim=10, cleantype='idw', verbose=False
    # )

    # create a figure with text on mouse hover
    fig = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], plot_width=p_width, plot_height=p_height,
        tools=[PanTool(),BoxZoomTool(),WheelZoomTool(),ResetTool(),HoverTool()])
    fig.x_range.range_padding = fig.y_range.range_padding = 0

    r = fig.multi_line('x', 'y', source={'x':[],'y':[]},color='white',line_width=3)
    fig.add_tools(FreehandDrawTool(renderers=[r]))

    # set up a colobar + data range
    color_mapper = LogColorMapper(palette="Cividis256", low=np.percentile(data, 55), high=np.percentile(data, 99))

    # must give a vector of image data for image parameter
    fig.image(
        image=[data],
          x=0, y=0, dw=hdu[0].data.shape[1], dh=hdu[0].data.shape[0],
          level="image", color_mapper=color_mapper
    )
    fig.grid.grid_line_width = 0.5

    color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
                         label_standoff=12, border_line_color=None, location=(0,0))

    fig.add_layout(color_bar, 'right')

    if sample_data:
      starfield = figure()
      starfield.image_url(['https://app.aavso.org/vsp/chart/X28194FDL.png'], 0, 1, 0.8, h=0.6)  
      show(row(fig,starfield))
    else:
      show(fig)

#########################################################

def floats_to_ints(l):
  while (True):
#    print (l)
    m = re.search(r"^(.*?)(\d+\.\d+)(.*?)$", l)
    if m:
      start, fl, end = m.group(1), float(m.group(2)), m.group(3)
      l = start+str("%.0f" % fl)+end
    else:
      return(l)
  
#########################################################

# Find a field in the image fits header or prompt the user to enter the corresponding
# value.

def check_dir(p):
  p = p.replace("\\", "/")

  if not(os.path.isdir(p)):
    print(f"Problem: the directory {p} doesn't seem to exist")
    print("on your Gdrive filesystem.")
    return("")
  return(p)

#########################################################

def add_sign(var):
  str_var = str(var)
  m=re.search(r"^[\+\-]", str_var)
  if m:
    return(str_var)
  if float(var) >= 0:
    return(str("+%.6f" % float(var)))
  else:
    return(str("-%.6f" % float(var)))

#########################################################

def get_val(hdr, ks):
  for key in ks:
    if key in hdr.keys():
      return hdr[key]
    if key.lower() in hdr.keys():
      return hdr[key.lower()]
    new_key = key[0]+key[1:len(key)].lower()  # first letter capitalized
    if new_key in hdr.keys():
      return hdr[new_key]
  return("")

#########################################################

def process_lat_long(val, key):
  m = re.search(r"\'?([+-]?\d+)[\s\:](\d+)[\s\:](\d+\.?\d*)", val)
  if m:
    deg, min, sec = float(m.group(1)), float(m.group(2)), float(m.group(3))
    if deg < 0:
      v = deg - (((60*min) + sec)/3600)
    else:
      v = deg + (((60*min) + sec)/3600)
    return(add_sign(v))
  m = re.search("^\'?([+-]?\d+\.\d+)", val)
  if m:
    v = float(m.group(1))
    return(add_sign(v))
  else:
    print(f"Cannot match value {val}, which is meant to be {key}.")

#########################################################

# Convert a MicroObservatory timestamp (which is in local time) to UTC.

def convert_Mobs_to_utc(datestamp, latitude, longitude, height):

#  print(datestamp)
  t = Time(datestamp[0:21], format='isot', scale='utc')
  t -= 0.33

  return(str(t)[0:10])

#########################################################

def find (hdr, ks, obs):
  # Special stuff for MObs and Boyce-Astro Observatories
  boyce = {"FILTER": "ip", "LATITUDE": "+32.6135", "LONGITUD": "-116.3334", "HEIGHT": 1405 }
  mobs = {"FILTER": "V", "LATITUDE": "+37.04", "LONGITUD": "-110.73", "HEIGHT": 2606 }

  if "OBSERVAT" in hdr.keys() and hdr["OBSERVAT"] == 'Whipple Observatory':
    obs = "MObs"

#  if "USERID" in hdr.keys() and hdr["USERID"] == 'PatBoyce':
#    obs = "Boyce"

  if obs == "Boyce":
    boyce_val = get_val(boyce, ks)
    if (boyce_val != ""):
      return(boyce_val)
  if obs == "MObs":
    mobs_val = get_val(mobs, ks)
    if (mobs_val != ""):
      return(mobs_val)

  val = get_val(hdr, ks)

  if ks[0] == "LATITUDE" and val != "":         # Because EXOTIC needs these with signs shown.
    return(process_lat_long(str(val), "latitude"))
  if ks[0] == "LONGITUD" and val != "":
    return(process_lat_long(str(val), "longitude"))

  if (val != ""):
    return(val)

  print(f"\nI cannot find a field with any of these names in your image header: \n{ks}.")
  print("Please enter the value (not the name of the header field, the actual value) that should")
  print("be used for the value associated with this field.\n")
  if ks[0] == "HEIGHT":
    print("The units of elevation are meters.")
  value = input("")
  return(value)

###############################################

def look_for_calibration(image_dir):
  darks_dir, flats_dir, biases_dir = "null", "null", "null"

  m = re.search(r"(.*?)(\d\d\d\d\-\d\d\-\d\d)\/images", image_dir)  # This handles the way I set up the MObs image paths for my seminar teams.
  if m:
    prefix, date = m.group(1), m.group(2)
    darks = prefix+date+"/darks"
    if os.path.isdir(darks):
      darks_dir = str("\""+darks+"\"")
      
  d_names = ["dark", "darks", "DARK", "DARKS", "Dark", "Darks"]  # Possible names for calibration image directories.
  f_names = ["flat", "flats", "FLAT", "FLATS", "Flat", "Flats"]
  b_names = ["bias", "biases", "BIAS", "BIASES", "Bias", "Biases"]

  for d in d_names:
    if os.path.isdir(os.path.join(image_dir, d)):
      darks_dir = str("\""+os.path.join(image_dir, d)+"\"")
      break

  for f in f_names:
    if os.path.isdir(os.path.join(image_dir, f)):
      flats_dir = str("\""+os.path.join(image_dir, f)+"\"")
      break

  for b in b_names:
    if os.path.isdir(os.path.join(image_dir, b)):
      biases_dir = str("\""+os.path.join(image_dir, b)+"\"")
      break

  return(darks_dir, flats_dir, biases_dir)

###############################################

# Writes a new inits file into the directory with the output plots.  This prompts
# for needed information that it cannot find in the fits header of the first image.

def make_inits_file(planetary_params, image_dir, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data):
  inits_file_path = output_dir+"inits.json"
  hdul = fits.open(first_image)
  hdr = dict(hdul[0].header)

  min, max = "null", "null"
  filter = find(hdr, ['FILTER', 'FILT'], obs)
  if filter == "w":
    filter = "PanSTARRS-w"
    min = "404"
    max = "846"
  if filter == "Clear":
    filter = "V"
  if filter == "ip":
    min = "690"
    max = "819"
  if filter == "EXO":
    filter = "CBB"
  if re.search(r"Green", filter, re.IGNORECASE):
    filter = "SG"
    
  date_obs = find(hdr,["DATE", "DATE_OBS", "DATE-OBS"], obs)
  date_obs = date_obs.replace("/", "_")
  longitude = find(hdr,['SITELONG', 'LONGITUD', 'LONG', 'LONGITUDE'],obs)
  latitude = find(hdr,['SITELAT', 'LATITUDE', 'LAT'],obs)
  height = int(find(hdr, ['HEIGHT', 'ELEVATION', 'ELE', 'EL', 'OBSGEO-H', 'ALT-OBS', 'SITEELEV'], obs))
  obs_notes = "N/A"
  sec_obs_code = "N/A"

  mobs_data = False
  # For MObs, the date is local rather than UTC, so convert.
  if "OBSERVAT" in hdr.keys() and hdr["OBSERVAT"] == 'Whipple Observatory':
    date_obs = convert_Mobs_to_utc(date_obs, latitude, longitude, height)
    weather = hdr["WEATHER"] 
    temps = float(hdr["TELTEMP"]) - float(hdr["CAMTEMP"])
    obs_notes = str("First image seeing %s (0: poor, 100: excellent), Teltemp - Camtemp %.1f.  These observations were conducted with MicroObservatory, a robotic telescope network managed by the Harvard-Smithsonian Center for Astrophysics on behalf of NASA's Universe of Learning. This work is supported by NASA under award number NNX16AC65A to the Space Telescope Science Institute." % (weather, temps))
    sec_obs_code = "MOBS"  
    mobs_data = True
  
  if aavso_obs_code == "":
      aavso_obs_code = "N/A"

  obs_date = date_obs[0:10]
  (darks_dir, flats_dir, biases_dir) = look_for_calibration(image_dir)

  with open(inits_file_path, 'w') as inits_file:
    inits_file.write("""
{
  %s,
    "user_info": {
            "Directory with FITS files": "%s",
            "Directory to Save Plots": "%s",
            "Directory of Flats": %s,
            "Directory of Darks": %s,
            "Directory of Biases": %s,

            "AAVSO Observer Code (N/A if none)": "%s",
            "Secondary Observer Codes (N/A if none)": "%s",

            "Observation date": "%s",
            "Obs. Latitude": "%s",
            "Obs. Longitude": "%s",
            "Obs. Elevation (meters)": %d,
            "Camera Type (CCD or DSLR)": "CCD",
            "Pixel Binning": "1x1",
            "Filter Name (aavso.org/filters)": "%s",
            "Observing Notes": "%s",

            "Plate Solution? (y/n)": "y",
            "Align Images? (y/n)": "y",

            "Target Star X & Y Pixel": %s,
            "Comparison Star(s) X & Y Pixel": %s
    },    
    "optional_info": {
            "Pixel Scale (Ex: 5.21 arcsecs/pixel)": null,
            "Filter Minimum Wavelength (nm)": %s,
            "Filter Maximum Wavelength (nm)": %s
    }
}
""" % (planetary_params, image_dir, output_dir, flats_dir, darks_dir, biases_dir, 
       aavso_obs_code, sec_obs_code, obs_date, latitude, longitude, height, filter, 
       obs_notes, targ_coords, comp_coords, min, max))

  print("\nWithin your folder of images, there is now a new folder called EXOTIC_output.")
  print("This folder contains an initialization file for EXOTIC called inits.json.")
  print("The same folder will contain the output files and images when EXOTIC finishes running.")

  if not mobs_data:  
    print(f"\nThe inits.json file currently says that your observatory latitude was {latitude} deg,")
    print(f"longitude was {longitude} deg, and elevation was {height}m.  \n")
    print("*** If any of these are incorrect, please change them in the inits.json file. ***")
    print("*** (Please make sure that Western longitudes have a negative sign! ***")
    print("*** TheSkyX sometimes stamps Western longitudes as positive; this needs to be switched! ***\n")

  print("\nIf you want to change anything in the inits file, please do that now.")
  print("When you are done, press enter to continue.")
  okay = input()

  return(inits_file_path)

#########################################################



In [5]:
# p is the name of the folder entered by the user.  Decide what to do based on what
# is found in the folder.

bokeh.io.output_notebook()
sample_data = False

print("BUSH data paths are 'boyceastro-data/2020d05d27PatBoyceWASPd80b' OR 'etc'")

p = input("Press enter to run the Hat-p-32b sample data, or enter a path as described above.")
if p == "":
  sample_data = True
  if os.path.isdir("sample-data/HatP32Dec202017"):
    print("Actually, it looks like you already have sample data, no need to download it again.")
  else:
    !git clone https://github.com/rzellem/EXOTIC_sampledata.git sample-data
  p = "sample-data/HatP32Dec202017"

p = check_dir(p)
output_dir = os.path.join(p, "EXOTIC_output/")    
print(output_dir)

Loading BokehJS ...

BUSH data paths are 'boyceastro-data/2020d05d27PatBoyceWASPd80b' OR 'etc'


Press enter to run the Hat-p-32b sample data, or enter a path as described above. /home/jupyter-chandru/scratch/2022_exos/HAT-P-32-b


/home/jupyter-chandru/scratch/2022_exos/HAT-P-32-b/EXOTIC_output/


In [ ]:
inits = []    # array of paths to any inits files found in the directory
files = [f for f in sorted(os.listdir(p)) if os.path.isfile(os.path.join(p, f))]
fits_count, first_image = 0, ""
for f in files:
  if re.search(r"\.f[itz]+s?\.?g?z?$", f, re.IGNORECASE):
    if first_image == "":
      first_image = os.path.join(p, f)
    fits_count += 1
  if re.search(r"\.json$", f, re.IGNORECASE):
    inits.append(os.path.join(p, f))
print(f"Found {fits_count} image files and {len(inits)} initialization files in the directory.")

if fits_count >= 19:                  # more than 20 images in the folder --> run EXOTIC on these images
  if len(inits) == 1:                 # one inits file exists
    inits_path = os.path.join(p, inits[0])
    with open(inits_path) as i_file:
      inits_data = i_file.read()
      d = json.loads(inits_data)
      targ_coords = d["user_info"]["Target Star X & Y Pixel"]
      comp_coords = d["user_info"]["Comparison Star(s) X & Y Pixel"]
      input_dir = d["user_info"]["Directory with FITS files"]
      if input_dir != p:
        print(f"The directory with fits files should be {p} but your inits file says {input_dir}.")
        print("This may or may not cause problems.  Just letting you know.")
      print(f"Coordinates from your inits file:\ntarget: {targ_coords}\ncomps: {comp_coords}")
      output_dir = d["user_info"]["Directory to Save Plots"]
      
  else:                               # no inits file: display image and prompt for target, comp coords;
                                      # then make an inits file and put it into the output directory (with the plots)
    print("There are either 0 or > 1 inits files in your image directory, so we'll make a new one.")
    print("Displaying first image (this can take a minute).")
    display_image(first_image)
    obs = ""
    print("Enter the coordinates of the target in the format [x, y]")
    print("Do NOT include decimals in the coordinates.")
    print("Be sure to include the square brackets and comma.")
    print("Target Coordinates: [x, y]")
    targ_coords = input()  
    print("\nEnter the coordinates of your comparison stars in the format [[x1, y1], [x2, y2]]")
    print("Be sure to include all square brackets and commas.")
    print("Do NOT include decimals in the coordinates.")    
    print("(E.g. if you only have one comp, you would enter [[x, y]])")
    print("Comparison Star Coordinates: [[x1, y1], [x2, y2]]")
    comp_coords = input()
    aavso_obs_code = ""
    if not sample_data:
      print("If you have an AAVSO Observer code, enter it here.")
      print("If not (or if you are not sure), just press enter.")
      aavso_obs_code = input()
#    print("If you want an alternate output directory, type it here.")
#    alt_out = input()
#    if (re.search(r"\w", alt_out)):
#      output_dir = alt_out
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)
    inits = [make_inits_file(planetary_params, p, output_dir, first_image, targ_coords, comp_coords, obs, aavso_obs_code, sample_data)]

  if not os.path.isdir(output_dir):    # Make the output directory if it does not exist already.
    os.mkdir(output_dir)
  output_dir_for_shell = output_dir.replace(" ", "\ ")

# At this point, we have made an inits file if needed and are ready to run it.
# Or, if there were < 20 images in the directory (meaning that the directory was 
# only contained inits files, we'll run those one-by-one.

print("Path to the inits file(s) that will be used:")

for inits_file in inits:
  print(inits_file)

num_inits = len(inits)

commands = []
for inits_file in inits:
  with open(inits_file) as i_file:
    inits_data = i_file.read()
    d = json.loads(inits_data)
    date_obs = d["user_info"]["Observation date"]
    planet = d["planetary_parameters"]["Planet Name"]
    output_dir = d["user_info"]["Directory to Save Plots"]
    if not os.path.isdir(output_dir):
      os.mkdir(output_dir)
    inits_file_for_shell = inits_file.replace(" ", "\\ ")
    run_exotic = str(f"exotic -red {inits_file_for_shell} -ov < no.txt")
    debug_exotic_run = str(f"!exotic -red \"{inits_file}\" -ov")

    commands.append({"inits_file_for_shell": inits_file_for_shell, "output_dir": output_dir, 
                      "planet": planet, "date_obs": date_obs, 
                      "run_exotic": run_exotic, "debug_exotic_run": debug_exotic_run
                      })
    print(f"{debug_exotic_run}")
    !eval "$run_exotic"

    lightcurve = os.path.join(output_dir,"FinalLightCurve_"+planet+"_"+date_obs+".png")
    fov = os.path.join(output_dir,"temp/FOV_"+planet+"_"+date_obs+"_LinearStretch.png")
    triangle = os.path.join(output_dir,"temp/Triangle_"+planet+"_"+date_obs+".png")
    print(f"lightcurve: {lightcurve}\nfov: {fov}\ntriangle: {triangle}\n")

    if not (os.path.isfile(lightcurve) and os.path.isfile(fov) and os.path.isfile(triangle)):
      print(f"Something went wrong with {planet} {date_obs}.\nCopy the command below into a new cell and run to find the error:\n{debug_exotic_run}\n")
      continue

    imageA = widgets.Image(value=open(lightcurve, 'rb').read())
    imageB = widgets.Image(value=open(fov, 'rb').read())
    hbox = HBox([imageB, imageA])
    display(hbox)
    display(Image(filename=triangle))


Found 142 image files and 0 initialization files in the directory.
There are either 0 or > 1 inits files in your image directory, so we'll make a new one.
Displaying first image (this can take a minute).
/home/jupyter-chandru/scratch/2022_exos/HAT-P-32-b/HATP-32171220013343.FITS


Enter the coordinates of the target in the format [x, y]
Do NOT include decimals in the coordinates.
Be sure to include the square brackets and comma.
Target Coordinates: [x, y]


 [423,286]



Enter the coordinates of your comparison stars in the format [[x1, y1], [x2, y2]]
Be sure to include all square brackets and commas.
Do NOT include decimals in the coordinates.
(E.g. if you only have one comp, you would enter [[x, y]])
Comparison Star Coordinates: [[x1, y1], [x2, y2]]


 [[406,210],[348,214]]


If you have an AAVSO Observer code, enter it here.
If not (or if you are not sure), just press enter.



Within your folder of images, there is now a new folder called EXOTIC_output.
This folder contains an initialization file for EXOTIC called inits.json.
The same folder will contain the output files and images when EXOTIC finishes running.

If you want to change anything in the inits file, please do that now.
When you are done, press enter to continue.


Path to the inits file(s) that will be used:
/home/jupyter-chandru/scratch/2022_exos/HAT-P-32-b/EXOTIC_output/inits.json
!exotic -red "/home/jupyter-chandru/scratch/2022_exos/HAT-P-32-b/EXOTIC_output/inits.json" -ov
    Done!
Checking exotethys database...
Checking ephemerides database...
Checking photometry database...
Checking catalogues database...

*************************************************************
Welcome to the EXOplanet Transit Interpretation Code (EXOTIC)
Version 2.1.1
*************************************************************


**************************
Complete Reduction Routine
**************************

Would you like Comparison Stars added automatically from AAVSO? (y/n): 
**************************
Starting Reduction Process
**************************


Getting the plate solution for your imaging file to translate pixel coordinates on the sky. 
Please wait....
WCS file creation successful.
Thinking ... DONE!

Here is the path to your plate solution: /home/

**The cell above will take between 15 minutes and an hour to run after you enter your path; please be patient.**  

If you see an error message and / or the Colaboratory logo at the top turns red, please post to #data-reduction on the Exoplanet Watch Slack channel.  

**Explanation of the plots that are displayed at the end of the run:**

The plot titled "FOV" is the "Field of View" of the images.  Your target is circled, and if EXOTIC used a comparison star, that is circled too.  (EXOTIC does a quick preliminary fit using each of your comps, and chooses the one that gives the cleanest lightcurve to use in the more time-consuming, more rigorous fitting routine.  Sometimes, though, the cleanest lightcurve results from not using *any* comps.  In that case, you will only see your target circled.)

The lightcurve shows the dip in light as the planet blocked the star during your observations.  The last plot shows what are called the "posteriors" of the fit, obtained by an algorithm that is called "nested sampling".  This shows how certain each of the parameters are.  The main parameter of interest is the transit midpoint, tmid.  You should see Gaussian distributions; if you don't, then it might be because the data were noisy.  

All of the graphs above, plus several other output files, are now saved to your output directory (if you used this Colaboratory to make the inits file, that is in the EXOTIC_output folder within the folder of images on your Google drive).  The textfile whose name starts with "AAVSO" is already in the correct format to be uploaded to the American Association of Variable Star Observers' [Exoplanet Database](https://www.aavso.org/apps/exosite/doc).  Uploading will allow other astronomers to use your data to plan their observing campaigns for your target, and will allow NASA to use your data to optimize space telescopes.  (However, please do not upload the sample data to AAVSO; that has already been done.)


**Note that you can use this same Colaboratory again to reduce your next transit!**

### <font color='red'>Cells below this line might be relevant if you have multiple reduced transits of a single target; otherwise, just ignore them.</font>

In [ ]:
#@title Display output from multiple runs.
# Displays the output for folders within the path that is passed in.

#COMP_STAR-XC={"ra": "43.70630577421983", "dec": "-10.864885209014435", "x": "231", "y": "295"}
#COMP_STAR-XC=null
########################################################

def get_comp(aavso_filepath):
  print(f"In get_comp: aavso_filepath {aavso_filepath}")
  found_comp = False
  with open(aavso_filepath, 'r') as aavso:
    print("Opened file")
    for line in aavso:
      m = re.search("#COMP_STAR-XC=null", line)   # Return empty strings if no comp star was used.
      if m:
        return("no comp,no comp,no comp,no comp")
      m = re.search("#COMP_STAR-XC=\{\"ra\"\: \"(\d+\.\d+)\", \"dec\"\: \"([+-]?\d+\.\d+)\", \"x\"\: \"(\d+)\", \"y\"\: \"(\d+)\"\}", line)
      if m:
        comp_ra, comp_dec, comp_x, comp_y = m.group(1), m.group(2), m.group(3), m.group(4)
        print (f"Got comp_ra {comp_ra}, comp_dec {comp_dec}, comp_x {comp_x}, comp_y {comp_y}")
        return(f"{comp_ra},{comp_dec},{comp_x},{comp_y}")
      m = re.search("#COMP_STAR-XC=\{\"ra\"\: null, \"dec\"\: null, \"x\"\: \"(\d+)\", \"y\"\: \"(\d+)\"\}", line)
      if m:
        comp_ra, comp_dec, comp_x, comp_y = "null", "null", m.group(1), m.group(2)
        return(f"{comp_ra},{comp_dec},{comp_x},{comp_y}")
      

########################################################

def make_fpp_header(fpp):
  header = ""
  header_fields = sorted(fpp.keys())
  for field in header_fields:
    header = header + field + ","
  header = header +"comp RA,comp Dec,comp x,comp y"
  return(header+"\n")

########################################################

def include_params(fpp, aavso_filepath):
  params = ""
  fields = sorted(fpp.keys())
  for field in fields:
    params = params + fpp[field] + ","
  params = params + get_comp(aavso_filepath)
  return(params+"\n")

########################################################

def get_midt_err(fpp):
  midt = 0.0
  if "Mid-Transit Time" in fpp.keys():
    midt = fpp["Mid-Transit Time"]
  elif "Mid-Transit Time (Tmid)" in fpp.keys():
    midt = fpp['Mid-Transit Time (Tmid)']
  m = re.search(r"^\d+\.\d+\s\+\/\-\s(\d+\.\d+)", midt)
  if m:
    midt_err = float(m.group(1))
    return(midt_err)
  else:
    print(f"PROBLEM!  Cannot find midtransit time error!!!\n  {fpp}")

########################################################

def display_output(p, minimum_residual_scatter, min_midt_err, p_param_dict):
  fpps = ""
  dirs = [d for d in sorted(os.listdir(p)) if os.path.isdir(os.path.join(p, d))]
  for d in dirs:
    print(f"Searching directory {d}")
    output_images, fov_descs, date, lc_path = [], [], "", ""
    files = [f for f in sorted(os.listdir(os.path.join(p, d))) if os.path.isfile(os.path.join(p, d, f))]
    for f in files:
      if f == "wcs.fits" or re.search(r"Observing_Statistics", f):
        continue
      m = re.search(r"([^_]+)_([^_]+)_([^_^\.]+)_?([^\.]*)\.(.+)", f)
      if m:
        desc, target, date, desc2, filetype = m.group(1), m.group(2), m.group(3), m.group(4), m.group(5)
        if (target != p_param_dict["Planet Name"]):
          continue
        fp = os.path.join(p,d,f)
        if desc == "FinalLightCurve" and filetype == "png":
          image = widgets.Image(value=open(fp, 'rb').read())
          output_images.append(image)
        if desc == "FinalParams":
          with open(fp) as o_file:
            final_params = o_file.read()
            params = json.loads(final_params)
            fpp = params["FINAL PLANETARY PARAMETERS"]
            pprint(fpp)
            if "Best Comparison Star" in fpp:
              del fpp["Best Comparison Star"]
            if "Optimal Annulus" in fpp:
              del fpp["Optimal Annulus"]
            if "Optimal Aperture" in fpp:
              del fpp["Optimal Aperture"]
            if "Optimal Method" in fpp:
              del fpp["Optimal Method"]
            if fpps == "":
              fpps = "folder," + make_fpp_header(fpp)

            scatter = fpp['Scatter in the residuals of the lightcurve fit is']
            m = re.search(r"(\d+\.\d+)", scatter)
            scatter = float(m.group(1))
            if scatter < minimum_residual_scatter and get_midt_err(fpp) < min_midt_err:
              fpps = fpps + os.path.join(p,d) + "," + include_params(fpp, os.path.join(p, d, "AAVSO_"+target+"_"+date+".txt"))
        if desc == "FOV" and filetype == "png":
          if (desc2 == "SquaredStretch"):
            image = widgets.Image(value=open(fp, 'rb').read())
            output_images.append(image)
            fov_descs.append(desc2)

    lc_first = output_images[::-1] 
    if (len(output_images)>0):
      print(f"{d} {date}, {fov_descs[::-1]}")
      hbox = HBox(lc_first)
      display(hbox)
      print("\n*********************************************************\n")

    fpps = fpps + display_output(os.path.join(p, d), minimum_residual_scatter, min_midt_err, p_param_dict)

  return(fpps)

########################################################

def reformat_header(header):
  header_list = header.split(',')
  new_header_list = []
  midt_field_index = -1
  for field in header_list:
    midt_field_index += 1
    if field == "folder" or field == "comp RA" or field == "comp Dec" or field == "comp x" or field == "comp y" or re.search(r'^Scatter', field) or re.search(r'Transit depth uncertainty', field):
      new_header_list.append(field)
      continue
    else:
      new_header_list.append(field)
      new_header_list.append(field+" ERROR")  
  new_header = ",".join(new_header_list)
  return(new_header, new_header_list)

##############################################

def reformat_fpp_list(fpps, t0, period):
  fpp_list = []
  orig_fpp_list = fpps.split('\n')
  orig_header = orig_fpp_list[0]
  for fpp in orig_fpp_list:
    if (fpp == orig_header):
      continue
    fpp_list.append(fpp)
  
  header, field_list = reformat_header(orig_header)
  
  for i in range(len(fpp_list)):
    fpp_list[i] = fpp_list[i].replace(" +/- ", ",")
    fp_list = fpp_list[i].split(',')
    for j in range(len(fp_list)):
      if j == 0:
        continue
      m = re.search(r'^(\d+\.?\d*)', fp_list[j])
      if m:
        fp_list[j] = m.group(1)
    fpp_list[i] = ",".join(fp_list)

  fpp_dicts = {}
  for i in range(len(fpp_list)):
    transit_fpp = fpp_list[i].split(',')
    folder = transit_fpp[0]
    if not re.search(r'\w', folder):
      continue
    fpp_dict = { "all_vals": transit_fpp }

    for j in range(len(field_list)):
      field = field_list[j]
      value = transit_fpp[j]
      fpp_dict[field] = value
    epoch = round((float(fpp_dict["Mid-Transit Time (Tmid)"]) - t0)/period)
    while epoch in fpp_dicts.keys():    #  Just in case the same transit is observed by 2 different telescopes.
      epoch += 0.00001
    fpp_dicts[epoch] = fpp_dict

  obs_midt_list, obs_midt_unc_list = [], []

  sorted_fpp_list = []
  for key in sorted(fpp_dicts.keys()):
    all_vals = fpp_dicts[key]["all_vals"]
    sorted_fpp_list.append(all_vals)
    obs_midt, obs_midt_unc = 0.0, 0.0
    if "Mid-Transit Time" in fpp_dicts[key].keys():
      obs_midt = fpp_dicts[key]["Mid-Transit Time"]
      obs_midt_unc = fpp_dicts[key]["Mid-Transit Time ERROR"]
    elif "Mid-Transit Time (Tmid)" in fpp_dicts[key].keys():
      obs_midt = fpp_dicts[key]['Mid-Transit Time (Tmid)']
      obs_midt_unc = fpp_dicts[key]["Mid-Transit Time (Tmid) ERROR"]
    obs_midt_list.append(obs_midt)
    obs_midt_unc_list.append(obs_midt_unc)

  return(header, sorted_fpp_list, fpp_dicts, obs_midt_list, obs_midt_unc_list)

##############################################

def observed_minus_calculated(midt, midt_unc, t0, t0_unc, period, period_unc):
  epoch = round((midt - t0)/period)

  calc = t0 + epoch*period    # Equation 1 from Zellem et. al., 2020.
  obs_minus_calc = midt - calc

  calc_unc = ((epoch**2) * (period_unc**2))
  calc_unc += (2 * epoch * period_unc * t0_unc)
  calc_unc += (t0_unc**2)
  calc_unc = math.sqrt(calc_unc)       # Equation 3 from Zellem et. al., 2020.

  obs_minus_calc_unc = math.sqrt(midt_unc**2 + calc_unc**2)   # sum the uncertainties of observed and calculated values in quadrature
  
  # Convert from days to minutes:
  obs_minus_calc = obs_minus_calc * 24 * 60
  obs_minus_calc_unc = obs_minus_calc_unc * 24 * 60

  return(epoch, obs_minus_calc, obs_minus_calc_unc)

#######################################################

def find_o_c(t0, t0_unc, period, period_unc, obs_midt_list, obs_midt_unc_list):
  epochs, o_c, o_c_unc = [], [], []
  for i in range(len(obs_midt_list)):
    midt, midt_unc = float(obs_midt_list[i]), float(obs_midt_unc_list[i])
    epoch, obs_minus_calc, obs_minus_calc_unc = observed_minus_calculated(midt, midt_unc, t0, t0_unc, period, period_unc)
    epochs.append(epoch)
    o_c.append(obs_minus_calc)
    o_c_unc.append(obs_minus_calc_unc)

  return(epochs, o_c, o_c_unc)

#######################################################

def get_plot_xrange (e):
  max_x = max(e)
  if max(e) > max_x:
    max_x = max(e)

  min_x = min(e)
  if min(e) < min_x:
    min_x = min(e)

  x_overflow = (max_x - min_x)/15
  min_x -= x_overflow
  max_x += x_overflow

  return(max_x, min_x, x_overflow)

###############################################################

def make_plot (planet, epoch, o_c, uncertainty):

  plot = bk.plotting.figure(
    plot_width=600, plot_height=400,
    title = "%s O - C" % (planet),
    x_axis_label = "Epoch (cycle)", y_axis_label = "O - C (min)", 
  )
  
  err_xs = []
  err_ys = []

  for x, y, yerr in zip(epoch, o_c, uncertainty):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

  max_x, min_x, x_overflow = get_plot_xrange (epoch)

  x_axis = [min_x, max_x]
  zeros = [0, 0]

  plot.x_range = Range1d(min_x, max_x)
  
  plot.multi_line(err_xs, err_ys, color='grey') 
  plot.scatter(epoch, o_c, size=6, color='blue')

  plot.line(x_axis, zeros, color='black')

  m,b = np.polyfit(epoch,o_c,1)
  x_vals = [min_x,max_x]
  y_vals = [b+m*min_x,b+m*max_x]
  y_avg = (y_vals[1]+y_vals[0])/2.0
  # remove the # from the next lines to add line of best fit, slope, and average y value across this plot
  # plot.line(x_vals,y_vals,color="blue")
  # print("slope: ",m,"\naverage y value: ",y_avg)

  plot.title.text_font_size = '14pt'
  plot.xaxis.axis_label_text_font_size = "14pt"
  plot.yaxis.axis_label_text_font_size = "14pt"
  return(plot)

##############################################

def obs_calc_plot (fpps, t0, t0_unc, period, period_unc):
  header, fpp_list, fpp_dicts, obs_midt_list, obs_midt_unc_list = reformat_fpp_list(fpps, t0, period)  # fpp_list now sorted by epoch

  epochs, o_c, o_c_unc = find_o_c(t0, t0_unc, period, period_unc, obs_midt_list, obs_midt_unc_list)

  bk.io.output_notebook()
  plot = make_plot(planet, epochs, o_c, o_c_unc)
  bk.plotting.show(plot)

  out_str = str(header+",epoch,o_c (min),o_c_unc (min),midt (BJD),midt_unc (BJD)\n")
  for i in range(len(fpp_list)):
    fp, epoch, o_c_val, o_c_unc_val, obs_midt_val, obs_midt_unc_val = fpp_list[i], epochs[i], o_c[i], o_c_unc[i], obs_midt_list[i], obs_midt_unc_list[i]
    all_transit_data = str(f"{','.join(fp)},{epoch},{o_c_val},{o_c_unc_val},{obs_midt_val},{obs_midt_unc_val}")
    out_str = out_str + all_transit_data + "\n"
  print(out_str)

  return(epochs, obs_midt_list, obs_midt_unc_list, fpp_list)

##############################################

# Kalee's folders:
# WASP-35b
# The Orbiteers/EXOTIC
# Wasp-43b/output
# 2021 Qatar-1b inits/EXOTIC_output
# 2021 Tres-1b inits/EXOTIC_output

import math
import bokeh as bk
import bokeh.io
import bokeh.plotting
from bokeh.models import Range1d, Label

planet = p_param_dict["Planet Name"]
t0 = float(p_param_dict["Published Mid-Transit Time (BJD-UTC)"])
t0_unc = float(p_param_dict["Mid-Transit Time Uncertainty"])
period = float(p_param_dict["Orbital Period (days)"])
period_unc = float(p_param_dict["Orbital Period Uncertainty"])

minimum_residual_scatter = 2.2   # This is the minimum scatter in the residuals of the lightcurve fit
                                 # that you will accept as a "decent" lightcurve.

min_midt_err = 0.007             # Similarly, this is the minimum midtransit time uncertainty that 
                                 # that you will accept as a "decent" lightcurve

print("This cell recursively searches the folder you input for EXOTIC output files\ncorresponding to the target you looked up in the second cell.\n")
print(f"It displays all lightcurves, FOV's, and final planetary parameters for that target.\n*If* the residual scatter is less than {minimum_residual_scatter}% and the midtransit time uncertainty\nis less than {min_midt_err}, it includes the final planetary parameters in the comma-\ndelimited list at the end, and includes the corresponding point on the O-C plot.\n")

p = input("Give the path to a directory that contains folders of EXOTIC output:")
if not re.search(r"^\/content\/drive\/My Drive", p):
  p = os.path.join("/content/drive/My Drive", p)
if not p[len(p)-1] == '/':
  p = p+"/"
print(p)

fpps = display_output(p, minimum_residual_scatter, min_midt_err, p_param_dict)
epochs, obs_midt_list, obs_midt_unc_list, fpp_list = obs_calc_plot(fpps, t0, t0_unc, period, period_unc)


In [ ]:
#@title <font color='red'>Optional</font> See existing Exoplanet Watch Lightcurves for this target 
from PIL import Image as Img
from IPython.display import Image
from exotic.api.ew import ExoplanetWatch
from matplotlib import image
from google.colab.patches import cv2_imshow

###################################################

def resize_image(src_img, size=(64,64), bg_color="white"): 
    src_img.thumbnail(size, Img.ANTIALIAS)
    new_image = Img.new("RGB", size, bg_color)
    new_image.paste(src_img, (int((size[0] - src_img.size[0]) / 2), int((size[1] - src_img.size[1]) / 2)))
    return new_image

###################################################

def get_EW_data(target):
  EW = ExoplanetWatch()
  result = EW.get(target)
  if result == {}:
    return(result)
  print(f"Here are the priors for {target}:")
  for prior in result.priors:
    val = result.priors[prior]
    print(f"{prior}: {val}")
  return(result)

###################################################

def download_lightcurves(EW_target_data):
  "Downloading the lightcurves (this might take a few minutes and spew unnecessary output) . . . "
  num_observations = len(EW_target_data.observations)
  for i in range(num_observations):
    url = EW_target_data.observations[i].lightcurve_url
    download_lc = str(f"curl -o lc{i}.png {url}")
    !eval "$download_lc"
    print(f"{url}")


###################################################

def display_lightcurves(EW_target_data, size, background_color):
  num_observations = len(EW_target_data.observations)
  for i in range(num_observations):
    obs = EW_target_data.observations[i]
    obscode = obs.obscode["id"]
    org = obs.obscode["org"]
    secondary_obscode = "N/A"
    url = obs.lightcurve_url
    if len(obs.secondary_obscodes) > 0:
      secondary_obscode = obs.secondary_obscodes[0]["id"]
    tmid = obs.parameters['tmid']
    t = Time(tmid, format='jd')
    print(f"observer: {obscode}, org {org}, secondary_observer: {secondary_obscode}, tmid: {t.iso}\nurl: {url}")
    img_path = str(f"lc{i}.png")
    img = Img.open(img_path)
    resized_img = np.array(resize_image(img, size, background_color))
    cv2_imshow(resized_img)

###################################################

size = (600,600)
background_color="white"
EW_target_data = get_EW_data(target)
if EW_target_data != {}:
  num_observations = len(EW_target_data.observation_count)
  print(f"\nThere are {num_observations} observations for {target} in the Exoplanet Watch database.\n")
  if (num_observations > 0):
    download_lightcurves(EW_target_data)
    display_lightcurves(EW_target_data, size, background_color)
    url=EW_target_data.ephemeris['ephemeris_url']
    download_lc = str(f"curl -o ephemeris.png {url}")
    !eval "$download_lc"
    print(f"{url}")
    img = Img.open('ephemeris.png')
    resized_img = np.array(resize_image(img, size, background_color))
    cv2_imshow(resized_img)
else:
  print(f"There are no observations for {target} on the Exoplanet Watch site.")
  print(f"It would be great if you could add some!  :)")
